## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [32]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import seaborn as sns
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
config = pn.Config('master', 'V', 15)

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data
Dataset "us-companies" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (1 days old).


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-income-banks-quarterly" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-income-insurance-quarterly" on disk (1 days old).


...

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-derived-banks-quarterly" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-derived-insurance-quarterly" on disk (1 days old).


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
There is an error in the supplied ticker symbol EEFT in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company uses non stardard quarterly reporting date not currently supported. Company was dropped from the list.
There is an error in the supplied ticker symbol PAGS in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company was not found and was dropped from the list.


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


In [59]:
c = pn.Comps(config)
c.run()
c.comp_ratios

In [ ]:
c.calc_fair_value()